In [15]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pmdarima.arima import auto_arima
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import itertools
import warnings
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#from tqdm import tqdm
#import itertools
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tpot import TPOTClassifier
from tpot import TPOTRegressor
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from textblob import TextBlob
import re
from typing import List
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier


LOCALLY

GitHub

In [34]:
# # condense into one df 
df1 = pd.read_csv('Datasets/aspergers.csv', parse_dates=True)
df2 = pd.read_csv('Datasets/depression.csv', parse_dates=True)
df3 = pd.read_csv('Datasets/ocd.csv', parse_dates=True)
df4 = pd.read_csv('Datasets/ptsd.csv', parse_dates=True)
df5 = pd.read_csv('Datasets/adhd.csv', parse_dates=True)

df = pd.concat([df1, df2, df3, df4,df5], ignore_index=True)
#LOWER ALL subreddit 
df['subreddit'] = df['subreddit'].str.lower()
df.head()

In [33]:
null_subreddit_count = df['subreddit'].isnull().sum()
print(null_subreddit_count)

df['subreddit'].unique()

array(['aspergers', 'depression', 'ocd', 'ptsd', 'adhd'], dtype=object)

Exported sample size of post for external regex validation 

In [41]:
df_testy = df['body'].sample(n=50, random_state=1)
df_testy_df = df_testy.to_frame()

(37109, 10)

Next we will need to hot encode text variables we want to include in the model. For example Body is a post that is text based will convert this in to a sentiment score 
Also subreddit will be our target variable so we need to preform mapping on this column and then excluded the columns we will not use

In [46]:
#clean the text before performing sentiment analysis my hope is to improve optimization and accuracy improvement

def clean_text(text: str, slang_map: dict = None, stopwords: List[str] = None) -> str:
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user mentions (@username)
    text = re.sub(r'\@\w+', '', text)
    # Remove hashtags (but keep the text)
    text = re.sub(r'\#\w+', '', text)
    # Remove HTML entities (e.g., &amp;, &lt;, etc.)
    text = re.sub(r'\&\w+;', '', text)
    # Remove emojis
    emoji_pattern = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Optional: Convert slang or shorthand notations to full form
    if slang_map:
        for slang, full_form in slang_map.items():
            text = text.replace(slang, full_form)
    # Optional: Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Optional: Remove stopwords
    if stopwords:
        text = ' '.join([word for word in text.split() if word not in stopwords])
    # Optional: Convert text to lowercase
    text = text.lower()
    return text

In [47]:
#mapping 
subreddit_to_target = {
    'aspergers': 1,
    'depression': 2,
    'ocd': 3,
    'ptsd': 4,
    'adhd' :5,
}

df['target'] = df['subreddit'].map(subreddit_to_target)

# hot coding body to sentiment_score 
df['clean_body'] = df['body'].apply(lambda x: clean_text(str(x)))

# Apply sentiment analysis
df['sentiment_score'] = df['clean_body'].apply(lambda text: TextBlob(text).sentiment.polarity)
#setting target varibale 
test_df = df[['target','num_comments','score','upvote_ratio','sentiment_score','target']]
test_df.head()

,target,num_comments,score,upvote_ratio,sentiment_score,target
0,1,0,1,1.0,0.412500,1
1,1,1,1,1.0,-0.063263,1
2,1,2,3,1.0,0.021373,1
3,1,3,1,1.0,0.013254,1
4,1,0,1,1.0,0.000000,1


In [48]:
null_counts = df[df['target'].isnull()]['subreddit'].value_counts()
print(null_counts)


Series([], Name: count, dtype: int64)


With multiple health status this is a multiclass classification problem. 

In [49]:
#FEatures
X = df[['num_comments', 'score', 'upvote_ratio', 'sentiment_score']]
#Target Variable
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Null validation feel free to skip

In [50]:
print(y_train.isnull().sum())
null_counts_by_subreddit = df[df['target'].isnull()].groupby('subreddit').size()
print(null_counts_by_subreddit)

0
Series([], dtype: int64)


I started with a wider net for model selection may narrow (expensivge model performance not optimal will refine sentiment )

In [9]:
tpot = TPOTClassifier(
    generations=10,  # More generations
    population_size=100,  # Larger population
    verbosity=2,
    random_state=42,
    scoring='accuracy'
)

tpot.fit(X_train, y_train)

print(f"Test Accuracy: {tpot.score(X_test, y_test)}")



                                                                                   
Generation 1 - Current best internal CV score: 0.44384354423647876
                                                                                    
Generation 2 - Current best internal CV score: 0.4449711733834686
                                                                                    
Generation 3 - Current best internal CV score: 0.44529957701082556
                                                                                   
Generation 4 - Current best internal CV score: 0.44621919454132913
                                                                                   
Generation 5 - Current best internal CV score: 0.44621919454132913
                                                                                   
Generation 6 - Current best internal CV score: 0.44621919454132913
                                                                                   
Generatio

In [10]:
#tpot.export('tpot_best_model.py') Export the best model

Not promising as of yet will work on improving sentiment and see if we can create more features

In [51]:
gradient_boosting_model = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=4,
    max_features=0.35,
    min_samples_leaf=12,
    min_samples_split=12,
    n_estimators=100,
    subsample=0.7,
    random_state=42  # Setting the random state for reproducibility
)

# Fit the model on the training data
gradient_boosting_model.fit(X_train, y_train)

# Predict on the test data
y_pred = gradient_boosting_model.predict(X_test)

# Evaluate the predictions, e.g., by calculating the accuracy
from sklearn.metrics import accuracy_score

test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.4104038601361623
